# 基于`LSTM模型`预测未来一个交易日`开盘价的涨跌`

## 1. 数据预处理
### 1.1 根据股票代码划分数据

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np

# 全局变量
cols = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20] # 选取属性
feanum=len(cols) - 3 # 特征数量
window=5 # 时间窗设置

/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
Duplicate key in file PosixPath('/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/matplotlib/mpl-data/matplotlibrc'), line 400 ('axes.unicode_minus: True  # use Unicode for the minus symbol rather than hyphen.  See')
/home/tianhao/env4tiho_py36/lib/python3.6/site-packages/pandas/compat/__init__.py:85: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


* **训练集**

In [2]:
train_df = pd.read_csv("../data/SharpBiClsData/trainData.csv", usecols=cols)
# train_df = pd.read_csv("../data/trainData.csv")
train_stocks_code = train_df["kdcode"].unique()
train_stocks_num = len(train_stocks_code)

"训练集股票数量：",train_stocks_num

('训练集股票数量：', 3604)

In [3]:
train_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label
0,2016-01-04,000004.SZ,0.032267,0.129146,0.031824,0.488994,0.488731,0.488527,0.488106,0.487706,...,0.025756,0.025756,0.025756,0.936769,0.936935,0.936933,0.936923,0.558879,0.299335,-0.183007
1,2016-01-05,000004.SZ,0.054882,0.199040,0.054132,0.442429,0.465433,0.465220,0.464779,0.464362,...,0.028880,0.028880,0.028880,0.764645,0.850856,0.850852,0.850829,0.391955,0.653598,0.029333
2,2016-01-06,000004.SZ,0.029884,0.109187,0.029474,0.445773,0.458782,0.458567,0.458121,0.457698,...,0.034311,0.034311,0.034311,0.787185,0.829678,0.829673,0.829647,0.211296,0.616759,-0.002591
3,2016-01-07,000004.SZ,0.006455,0.021836,0.006364,0.412143,0.433148,0.446824,0.446368,0.445936,...,0.034308,0.032170,0.032170,0.785136,0.779124,0.818571,0.818544,0.342300,0.382222,-0.087532
4,2016-01-08,000004.SZ,0.043489,0.141737,0.042894,0.396876,0.417954,0.436721,0.436257,0.435817,...,0.038314,0.034540,0.034540,0.716081,0.762933,0.798096,0.798065,0.503942,0.602638,-0.023627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822607,2019-12-25,688389.SH,0.048334,0.074939,0.115043,0.185098,0.181560,0.183117,0.180226,0.177183,...,0.042325,0.042269,0.043423,0.053311,0.053109,0.053716,0.052761,0.009690,0.581775,0.009146
2822608,2019-12-26,688389.SH,0.049122,0.076895,0.116914,0.186948,0.183438,0.183236,0.180975,0.177456,...,0.046297,0.043617,0.044086,0.053833,0.052958,0.053416,0.052865,0.008972,0.584694,0.018127
2822609,2019-12-27,688389.SH,0.042794,0.066764,0.101831,0.186296,0.185652,0.183075,0.182022,0.177687,...,0.043701,0.042475,0.043696,0.054877,0.054002,0.053577,0.053161,0.011664,0.576456,-0.035608
2822610,2019-12-30,688389.SH,0.027754,0.042335,0.066054,0.181994,0.184616,0.183210,0.182501,0.177728,...,0.042571,0.043833,0.043392,0.052789,0.053828,0.053277,0.053318,0.009331,0.582087,0.005538


In [4]:
# 根据股票代码划分数据
for i, stock_i in enumerate(tqdm(train_stocks_code)):
    stock_i_data = train_df[train_df['kdcode'].isin([stock_i])]
    exec("train_df%s = stock_i_data" % i)

100%|██████████| 3604/3604 [01:53<00:00, 31.64it/s]


* **测试集**

In [5]:
test_df = pd.read_csv("../data/SharpBiClsData/testData.csv", usecols=cols)
# test_df = pd.read_csv("../data/testData.csv")
test_stocks_code = test_df["kdcode"].unique()
test_stocks_num = len(test_stocks_code)

"测试集股票数量：",test_stocks_num

('测试集股票数量：', 3790)

In [6]:
# 根据股票代码划分数据
for i, stock_i in enumerate(tqdm(test_stocks_code)):
    stock_i_data = test_df[test_df['kdcode'].isin([stock_i])]
    exec("test_df%s = stock_i_data" % i)

100%|██████████| 3790/3790 [00:56<00:00, 67.42it/s]


### 1.2. 将原始数据改造为LSTM网络的输入
分割出window个时间窗的数据为输入的`X`

紧接着的那条数据为标签`Y`

因此需要将每只股票的数据按照时间(日期)分割成`window + 1`长度的数据

* **训练集**

In [7]:
train_df

,dt,kdcode,volume,turnover,turn,vwap,vwap_mean3,vwap_mean5,vwap_mean10,vwap_mean20,...,chg_mean3,chg_mean5,chg_mean10,adj_open,open_mean3,open_mean5,open_mean10,high_low,close_open,label
0,2016-01-04,000004.SZ,0.032267,0.129146,0.031824,0.488994,0.488731,0.488527,0.488106,0.487706,...,0.025756,0.025756,0.025756,0.936769,0.936935,0.936933,0.936923,0.558879,0.299335,-0.183007
1,2016-01-05,000004.SZ,0.054882,0.199040,0.054132,0.442429,0.465433,0.465220,0.464779,0.464362,...,0.028880,0.028880,0.028880,0.764645,0.850856,0.850852,0.850829,0.391955,0.653598,0.029333
2,2016-01-06,000004.SZ,0.029884,0.109187,0.029474,0.445773,0.458782,0.458567,0.458121,0.457698,...,0.034311,0.034311,0.034311,0.787185,0.829678,0.829673,0.829647,0.211296,0.616759,-0.002591
3,2016-01-07,000004.SZ,0.006455,0.021836,0.006364,0.412143,0.433148,0.446824,0.446368,0.445936,...,0.034308,0.032170,0.032170,0.785136,0.779124,0.818571,0.818544,0.342300,0.382222,-0.087532
4,2016-01-08,000004.SZ,0.043489,0.141737,0.042894,0.396876,0.417954,0.436721,0.436257,0.435817,...,0.038314,0.034540,0.034540,0.716081,0.762933,0.798096,0.798065,0.503942,0.602638,-0.023627
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2822607,2019-12-25,688389.SH,0.048334,0.074939,0.115043,0.185098,0.181560,0.183117,0.180226,0.177183,...,0.042325,0.042269,0.043423,0.053311,0.053109,0.053716,0.052761,0.009690,0.581775,0.009146
2822608,2019-12-26,688389.SH,0.049122,0.076895,0.116914,0.186948,0.183438,0.183236,0.180975,0.177456,...,0.046297,0.043617,0.044086,0.053833,0.052958,0.053416,0.052865,0.008972,0.584694,0.018127
2822609,2019-12-27,688389.SH,0.042794,0.066764,0.101831,0.186296,0.185652,0.183075,0.182022,0.177687,...,0.043701,0.042475,0.043696,0.054877,0.054002,0.053577,0.053161,0.011664,0.576456,-0.035608
2822610,2019-12-30,688389.SH,0.027754,0.042335,0.066054,0.181994,0.184616,0.183210,0.182501,0.177728,...,0.042571,0.043833,0.043392,0.052789,0.053828,0.053277,0.053318,0.009331,0.582087,0.005538


In [8]:
# 分割数据
trainResult = []
for i in tqdm(range(train_stocks_num)): # 遍历训练集所有股票的DataFrame
    exec("trainData = train_df%s.values" % i)
    sequence_length = window + 1
    for index in range(len(trainData) - sequence_length + 1):
        prd = trainData[index: index + sequence_length]
        if abs(prd[-1, -1]) > 0.05:
            if prd[-1, -1] > 0:
                prd[-1, -1] = 0
            else:
                prd[-1, -1] = 1
            trainResult.append(prd)

trainResult = np.array(trainResult)
trainResult.shape

100%|██████████| 3604/3604 [00:03<00:00, 1010.22it/s]


(207346, 6, 21)

In [9]:
# 保存分割后每条数据的日期
train_date = trainResult[:,-1,0]
len(train_date)

207346

* **测试集**

In [10]:
testResult = []
for i in tqdm(range(test_stocks_num)): # 遍历训练集所有股票的DataFrame
    exec("testData = test_df%s.values" % i)
    sequence_length = window + 1
    for index in range(len(testData) - sequence_length + 1):
        prd = testData[index: index + sequence_length]
        if abs(prd[-1, -1]) > 0.05:
            if prd[-1, -1] > 0:
                prd[-1, -1] = 0
            else:
                prd[-1, -1] = 1
            testResult.append(prd)

testResult = np.array(testResult)
testResult.shape

100%|██████████| 3790/3790 [00:01<00:00, 2500.88it/s]


(117859, 6, 21)

In [11]:
# 保存分割后每条数据的日期
test_date = testResult[:,-1,0]
len(test_date)

117859

**分割出X**

In [12]:
X_train = trainResult[:, :-1, 2:-1]
X_test = testResult[:, :-1, 2:-1]
X_train = X_train.astype('float64')
X_test = X_test.astype('float64')
print("训练集X：" + str(X_train.shape))
print("测试集X：" + str(X_test.shape))

训练集X：(207346, 5, 18)
测试集X：(117859, 5, 18)


**分割出Y**

In [13]:
Y_train = trainResult[:, window, -1]
Y_train.shape

(207346,)

In [14]:
Y_test = testResult[:, window, -1]
Y_test.shape

(117859,)

In [15]:
Y_train = Y_train.astype('float64')
Y_test = Y_test.astype('float64')
# Y_train[0:5]
# Y_test[0:5]

## 3. 模型构建与训练

In [16]:
from sklearn.metrics import mean_squared_error
from sklearn import metrics
from math import sqrt
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.layers.recurrent import GRU
from keras.callbacks import EarlyStopping
import tensorflow as tf
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
for gpu in tf.config.experimental.list_physical_devices('GPU'):
    tf.config.experimental.set_memory_growth(gpu, True)

#建立、训练模型过程
d = 0.001
model = Sequential()#建立层次模型
model.add(LSTM(64, input_shape=(window, feanum), return_sequences=True))#建立LSTM层
model.add(Dropout(d))#建立的遗忘层
model.add(LSTM(32, input_shape=(window, feanum), return_sequences=False))#建立LSTM层
model.add(Dropout(d))#建立的遗忘层

model.add(Dense(16,kernel_initializer='uniform',activation='relu'))   #建立全连接层     
model.add(Dense(1, kernel_initializer = "uniform", activation = "sigmoid"))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, Y_train, epochs = 30, batch_size = 512, validation_split = 0.1) #训练模型epochs次

Epoch 1/30
365/365 [==============================] - 35s 37ms/step - loss: 0.6922 - accuracy: 0.5082 - val_loss: 0.6706 - val_accuracy: 0.5559
Epoch 2/30
123/365 [=========>....................] - ETA: 7s - loss: 0.6846 - accuracy: 0.5253

In [ ]:
#总结模型
model.summary()

## 4. 模型训练结果
* 训练集

In [ ]:
#在训练集上的拟合结果
Y_train_predict = model.predict(X_train)[:,0]
Y_train = Y_train

In [ ]:
import matplotlib.pyplot as plt

draw=pd.concat([pd.DataFrame(Y_train),pd.DataFrame(Y_train_predict)],axis=1)
draw.iloc[:300,0].plot(figsize=(12,6))
draw.iloc[:300,1].plot(figsize=(12,6))
plt.legend(('real', 'predict'),loc='upper right',fontsize='15')
plt.title("Train Data",fontsize='30') #添加标题
#展示在训练集上的表现

* 测试集

In [ ]:
#在测试集上的预测
Y_test_predict=model.predict(X_test)[:,0]
Y_test=Y_test

In [ ]:
draw=pd.concat([pd.DataFrame(Y_test),pd.DataFrame(Y_test_predict)],axis=1);
draw.iloc[:100,0].plot(figsize=(12,6))
draw.iloc[:100,1].plot(figsize=(12,6))
plt.legend(('real', 'predict'),loc='upper right',fontsize='15')
plt.title("Test Data",fontsize='30') #添加标题
# 展示在测试集上的表现

In [ ]:
txt = np.zeros(len(Y_test))
test_predict = Y_test_predict.copy()
test_predict[Y_test_predict > 0.5] = 1
test_predict[Y_test_predict <= 0.5] = 0

train_predict = Y_train_predict.copy()
train_predict[Y_train_predict > 0.5] = 1
train_predict[Y_train_predict <= 0.5] = 0

result = metrics.accuracy_score(Y_test, test_predict)
'测试集正确率:',result

* 训练过程Loss,Accuracy的变化

In [ ]:
acc = history.history['accuracy']
loss = history.history['loss']
epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'g', label='Training loss')
plt.plot(epochs, acc, 'r', label='Training acc')  
plt.title('Training loss and  acc')
plt.xlabel('Epochs')
plt.ylabel('Acc/Loss')
plt.legend()

* ROC曲线以及AUC值

In [ ]:
# 测试集
Y_test
Y_test_predict
fpr, tpr, thresholds = metrics.roc_curve(Y_test, Y_test_predict, pos_label=1)
auc = metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (area = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of test set')
plt.legend(loc="lower right")
plt.show()

In [ ]:
# 训练集
fpr, tpr, thresholds = metrics.roc_curve(Y_train, Y_train_predict, pos_label=1)
auc = metrics.auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
         lw=lw, label='ROC curve (auc = %0.2f)' % auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC of train set')
plt.legend(loc="lower right")
plt.show()

* f1-score

In [ ]:
print(metrics.f1_score(Y_train, train_predict))
print(metrics.f1_score(Y_test, test_predict))

* 混淆矩阵

In [ ]:
metrics.confusion_matrix(Y_train, train_predict)

In [ ]:
metrics.confusion_matrix(Y_test, test_predict)

* 显示主要分类指标

In [ ]:
print(metrics.classification_report(Y_train,train_predict))

In [ ]:
print(metrics.classification_report(Y_test,test_predict))

## 5. 保存模型

In [ ]:
model.save('./models/sharpModel_D1.h5')

## 6. 保存预测结果

In [ ]:
testResult.shape

In [ ]:
testResult[:,-1,:2]

In [ ]:
len(Y_test_predict)

In [ ]:
pred_df = pd.DataFrame(testResult[:, -1, :2], columns = ['dt', 'kdcode'])

In [ ]:
pred_df

In [ ]:
pred_df['score'] = Y_test_predict

In [ ]:
pred_df

In [ ]:
st_date = pred_df['dt'].min()
st_date

In [ ]:
dr = pd.date_range(st_date,'2021-7-20',freq='B') # 工作日
dr

In [ ]:
for date_i in tqdm(dr):
    dt = str(date_i.date())
    tmp_df = pred_df[pred_df['dt'] == dt]
    save_path = '../result4tiho/sharpModel_D1/prediction/' + dt + '.csv'
    tmp_df.set_index(tmp_df['kdcode'])
    tmp_df.to_csv(save_path, index = False)